In [1]:
import yaml
from apphub_configurator.models import *
import os
from pathlib import Path

current_dir = Path(os.getcwd())
parent_dir = current_dir.parent
current_dir

PosixPath('/home/t2/Desktop/p/EOEPCA/application-hub-context/config-generator/examples')

In [2]:
localstack_manifest_path = os.path.join(parent_dir, "manifests/manifest.yaml") 
with open(localstack_manifest_path, "r") as f:
    content = yaml.safe_load_all(f.read())


localstack_manifest = Manifest(
    name="manifests",
    key="manifests",
    readonly=True,
    persist=False,
    content=[e for e in content],
)

## Configuration



In [3]:
storage_class_rwo = "standard"
storage_class_rwx = "standard"

workspace_volume_size = "50Gi"
calrissian_volume_size = "50Gi"

node_selector = {}  # "k8s.scaleway.com/pool-name": "application-hub"}

## Volumes

Create the Volumes

### Workspace Volume

The workspace volume is persisted.

In [4]:
workspace_volume = Volume(
    name="workspace-volume",
    size=workspace_volume_size,
    claim_name="workspace-claim",
    mount_path="/workspace",
    storage_class=storage_class_rwo,
    access_modes=["ReadWriteOnce"],
    volume_mount=VolumeMount(name="workspace-volume", mount_path="/workspace"),
    persist=True,
)

### Calrissian Volume

This is a RWX volume, not persisted

In [5]:
calrissian_volume = Volume(
    name="calrissian-volume",
    claim_name="calrissian-claim",
    size=calrissian_volume_size,
    storage_class=storage_class_rwx,
    access_modes=["ReadWriteMany"],
    volume_mount=VolumeMount(name="calrissian-volume", mount_path="/calrissian"),
    persist=False,
)

## ConfigMaps

These configmaps are mounted as files on the pod.

### bash login

This file is used for the JupyterLab Terminal configuration

In [6]:
bash_login_file_path = os.path.join(parent_dir, "config-maps/bash-login") 
with open(bash_login_file_path, "r") as f:
    content = f.read()

bash_login_cm = ConfigMap(
    name="bash-login",
    key="bash-login",
    content=content,
    readonly=True,
    persist=True,
    mount_path="/workspace/.bash_login",
)

#### bash.rc


In [7]:
bash_rc_cm_file_path = os.path.join(parent_dir, "config-maps/bash-rc") 
with open(bash_rc_cm_file_path, "r") as f:
    content = f.read()
bash_rc_cm = ConfigMap(
    name="bash-rc",
    key="bash-rc",
    content=content,
    readonly=True,
    persist=True,
    mount_path="/workspace/.bashrc",
)

## Profiles

In [8]:
profiles = []

### Coder

In [9]:
coders = {
    "coder1": {
        "display_name": "Code Server Small",
        "slug": "eoepca_coder_slug_s",
        "cpu_limit": 2,
        "mem_limit": "8G",
    },
    "coder2": {
        "display_name": "Code Server Medium",
        "slug": "eoepca_coder_slug_m",
        "cpu_limit": 4,
        "mem_limit": "12G",
    },
}

for key, value in coders.items():
    coder_definition = ProfileDefinition(
        display_name=value["display_name"],
        slug=value["slug"],
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_limit=value["cpu_limit"],
            mem_limit=value["mem_limit"],
            image="eoepca/pde-code-server:develop",
        ),
    )

    coder_profile = Profile(
        id=f"profile_studio_{key}",
        groups=["group-a", "group-b"],
        definition=coder_definition,
        node_selector=node_selector,
        volumes=[calrissian_volume, workspace_volume],
        config_maps=[
            bash_rc_cm,
        ],
        pod_env_vars={
            "HOME": "/workspace",
            "CONDA_ENVS_PATH": " /workspace/.envs",
        },
        manifests=[localstack_manifest],
        env_from_config_maps=["my-config"],
        env_from_secrets=["my-secret", "data-by-name"],
        secret_mounts=[
            SecretMount(
                name="aws-credentials-{{ spawner.user.name }}",
                mount_path="/workspace/.aws",
            ),
            SecretMount(name="data-by-name", mount_path="/workspace/.data-by-name"),
            SecretMount(name="eoepca-plus-secret-ro", mount_path="/workspace/.docker"),
        ],
        image_pull_secrets=[ImagePullSecret(name="eoepca-plus-secret-ro")],
    )

    profiles.append(coder_profile)

## init.sh script ...

In [10]:
init_cm_file_path = os.path.join(parent_dir, "config-maps/init.sh") 
with open(init_cm_file_path, "r") as f:
    content = f.read()

init_cm = ConfigMap(
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
    default_mode="0660",
)


init_context_volume_mount = InitContainerVolumeMount(
    mount_path="/opt/init/.init.sh", name="init", sub_path="init"
)
init_container = InitContainer(
    name="init-file-on-volume",
    image="eoepca/pde-code-server:develop",
    command=["sh", "-c", "sh /opt/init/.init.sh"],
    volume_mounts=[
        VolumeMount(name="workspace-volume", mount_path="/workspace"),
        init_context_volume_mount,
    ],
)

eoepca_demo_init_script_profile = Profile(
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector=node_selector,
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)

##### Better code

In [11]:
profiles.append(eoepca_demo_init_script_profile)

eoepca_demo_init_script_profile = Profile(
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector=node_selector,
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)

## JupyterLab

In [12]:
image = "jupyter/scipy-notebook"


eoepca_jupyter_lab_profile = Profile(
    id="profile_jupyter_lab",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab",
        description="Jupyter Lab with Python 3.11",
        slug="eoepca_jupyter_lab",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector=node_selector,
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
)

profiles.append(eoepca_jupyter_lab_profile)

## Image pull secret


In [13]:
image_pull_secret = ImagePullSecret(
    name="cr-config",
    persist=False,
    data="",
)

In [14]:
with open(localstack_manifest_path, "r") as f:
    content = yaml.safe_load_all(f.read())


localstack_manifest = Manifest(
    name="manifests",
    key="manifests",
    readonly=True,
    persist=False,
    content=[e for e in content],
)

In [15]:
image = "jupyter/scipy-notebook"


eoepca_jupyter_lab_profile_2 = Profile(
    id="profile_jupyter_lab_2",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab - profile 2",
        description="Jupyter Lab with Python 3.11 - demoes the use of an image pull secret",
        slug="eoepca_jupyter_lab_2",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector=node_selector,
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
    image_pull_secrets=[image_pull_secret],
    manifests=[localstack_manifest],
)

profiles.append(eoepca_jupyter_lab_profile_2)

## Configuration

In [16]:
config = Config(profiles=profiles)
config_file_path = str(Path(current_dir).parent.parent / 'files' / 'hub' / 'config.yml')

with open(config_file_path, "w") as file:
    yaml.dump(config.dict(), file)

/tmp/ipykernel_1697007/2113836574.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  yaml.dump(config.dict(), file)


In [17]:
# profiles